#  How to automate Excel using Python

***

# 0. Install Dependencies

In [ ]:
!pip install pandas 
!pip install openpyxl --quiet
!pip install plotly-express --quiet
!pip install xlwings --quiet

# 1. Merge Excel Files (Simple)

### Merge multiple Excel files, perform calculations & export an interactive chart

In [ ]:
from pathlib import Path
import pandas as pd  # pip install pandas
import plotly.express as px  # pip install plotly-express

In [ ]:
# Locate examples files
INPUT_DIR = Path.cwd() / "1-Merge-Excel-Files-Simple" / "INPUT"
OUTPUT_DIR = Path.cwd() / "1-Merge-Excel-Files-Simple"

In [ ]:
# Create empty dataframe, iterate over input directory, append all excel files to empty dataframe
df = pd.DataFrame()

# If you want to iterate over all folders including subfolders, use: 
# for file in INPUT_DIR.rglob("*"):

for file in INPUT_DIR.iterdir():
    if file.suffix == ".xlsx":
        df_tmp = pd.read_excel(file)
        df = pd.concat([df, df_tmp], ignore_index=True)

df['Country'].unique()

**Please note**<br>
With pandas version 1.4.0 DataFrame.append() and Series.append() have been deprecated and will be removed in a future version.<br>
Hence, I have changed the code as follows to merge all Excel files into one DataFrame:
```diff
- df = df.append(pd.read_excel(file), ignore_index=True)

+ df_tmp = pd.read_excel(file)
+ df = pd.concat([df, df_tmp], ignore_index=True)
```

In [ ]:
df.head()

In [ ]:
df = df.groupby(by="Country").sum()[[" Sales", "Profit"]]
df

In [ ]:
df["Profit Margin %"] = (df["Profit"] / df[" Sales"]) * 100
df

In [ ]:
# Plot Profit situation & export grouped dataframe to Excel
fig = px.bar(
    df,
    x=df.index,
    y=" Sales",
    color="Profit Margin %",
    color_continuous_scale=["red", "yellow", "green"],
    template="plotly_white",
    title="<b>Profit Analysis</b>",
)
fig.write_html(str(OUTPUT_DIR / "profit_analysis.html"))
df.to_excel(OUTPUT_DIR / "summary.xlsx")
fig

### Remarks:

- Making use of the different plotting libraries, e.g. interactive visualizations using Plotly
- **Simplified Example** | In real-life, more often than not, datasets are not so well-structured

# 2. Merge Excel Files (Real Life Example)

### Merge multiple Excel files, perform calculations & insert (Excel) charts [keep excel formatting & formulas]

In [ ]:
from pathlib import Path
import pandas as pd  # pip install pandas
import xlwings as xw  # pip install xlwings

In [ ]:
# Locate examples files
INPUT_DIR = Path.cwd() / "2-Merge-Excel-Files-Real-Life" / "INPUT"
OUTPUT_DIR = Path.cwd() / "2-Merge-Excel-Files-Real-Life"

In [ ]:
# Create an empty workbook. This wb will be our final output
summary_wb = xw.Book()
summary_sht = summary_wb.sheets[0]
summary_sht.name = "Summary"

In [ ]:
# Iterate over each file in directory. Copy cell range & paste it into summary workbook (keep formatting & formuals)

# If you want to iterate over all folders including subfolders, use: 
# for file in INPUT_DIR.rglob("*"):

for file in INPUT_DIR.iterdir():
    if file.suffix == ".xlsx":
        wb = xw.Book(file)
        wb.app.display_alerts = False
        #wb.app.visible = False
        sht = wb.sheets("Overview")
        values = sht.range("B5").expand().copy()
        dest_last_row = summary_sht.range("A1").expand().last_cell.row + 1
        summary_sht.range(f"A{dest_last_row}").paste()
        wb.close()

In [ ]:
# Insert & style header of our summary workbook
header = [
    "Segment",
    "Country",
    "Product",
    "Discount Band",
    "Units Sold",
    "Sale Price",
    "Gross Sales",
    "Discounts",
    "Sales",
    "COGS",
    "Profit",
]
header_row = summary_sht.range("A1:K1")
header_row.value = header
header_row.font.bold = True
header_row.font.color = (255, 255, 255)
header_row.color = (119, 136, 153)

In [ ]:
# Use pandas to perform calculations, e.g. data aggregation
df = summary_sht.range("A1").options(pd.DataFrame, expand="table").value
data_by_country = df.groupby(by="Country").sum()[["Sales"]]
data_by_country

In [ ]:
# Insert grouped dataframe into our summary workbook
summary_sht.range("M1").value = data_by_country

In [ ]:
# Insert EXCEL chart
chart = summary_sht.charts.add(
    left=summary_sht.range("M8").left, 
    top=summary_sht.range("M8").top,
    width=400,
    height=200,
)
chart.set_source_data(summary_sht.range("M1").expand())
chart.chart_type = "bar_stacked"

In [ ]:
# Insert PANDAS chart
ax = data_by_country.sort_values(by="Sales").plot(kind="barh")
fig = ax.get_figure()
summary_sht.pictures.add(
    fig,
    name="Sales",
    update=True,
    left=summary_sht.range("M23").left,
    top=summary_sht.range("M23").top,
    width=400,
    height=200,
)

In [ ]:
# Clean up workbook, save workbook & quit Excel instance (if it is the only wb open)
summary_sht.autofit()
summary_wb.save(OUTPUT_DIR / "summary.xlsx")
if len(summary_wb.app.books) == 1:
    summary_wb.app.quit()
else:
    summary_wb.close()